In [30]:
import pandas as pd
import os
from IPython.display import display, Markdown, HTML
%matplotlib inline
import tkinter as tk
from tkinter import filedialog, messagebox
from openpyxl import load_workbook
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

''

In [31]:
messagebox.showinfo('OnSSET', 'Open the input file with calibrated GIS data')
input_file = filedialog.askopenfilename()
df = pd.read_csv(input_file)

In [32]:
df.sort_values(by=['Pop2018'], ascending=False, inplace=True)
cumulative_pop = df['Pop2018'].cumsum()/df['Pop2018'].sum() 

In [33]:
tier_1 = 12.5  # kWh/household/year
tier_2 = 204
tier_3 = 596
tier_4 = 1891
tier_5 = 3761

In [34]:
tier_split_2025 = {5: 0.03, # Share of population in Tier 5 
                   4: 0.16, # Share of population in Tier 5 or Tier 4
                   3: 0.16, # Share of population in Tier 5 or Tier 4 or Tier 3
                   2: 0.32, # Share of population in Tier 5 or Tier 4 or Tier 2
                   1: 1} # Share of population in Tier 5 or Tier 4 or Tier 2 or Tier 1 (Always = 1)

In [35]:
tier_split_2030 = {5: 0.03,
                   4: 0.20,
                   3: 0.21,
                   2: 0.37,
                   1: 1}

In [36]:
tier_split_2040 = {5: 0.04,
                   4: 0.26,
                   3: 0.37,
                   2: 0.58,
                   1: 1}

In [37]:
tier_split_2050 = {5: 0.05,
                   4: 0.35,
                   3: 0.87,
                   2: 1,
                   1: 1}

In [38]:
tier_split_2060 = {5: 0.07,
                   4: 0.40,
                   3: 0.99,
                   2: 1,
                   1: 1}

In [39]:
tier_split_2070 = {5: 0.07,
                   4: 0.40,
                   3: 0.99,
                   2: 1,
                   1: 1}

In [40]:
years = [2025, 2030, 2040, 2050, 2060, 2070]
tier_splits = {2025: tier_split_2025,
               2030: tier_split_2030,
               2040: tier_split_2040,
               2050: tier_split_2050,
               2060: tier_split_2060,
               2070: tier_split_2070}

In [41]:
for year in years:
    tier_split = tier_splits[year]
    
    # Tier 1
    df.loc[df['IsUrban'] == 2, 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_1 / 4.4
    df.loc[df['IsUrban'] < 2, 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_1 / 5.2
    df['Tier' + "{}".format(year)] = 1
    # Tier 2
    df.loc[(df['IsUrban'] == 2) & (cumulative_pop <= tier_split[2]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_2 / 4.4
    df.loc[(df['IsUrban'] < 2) & (cumulative_pop <= tier_split[2]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_2 / 5.2
    df.loc[cumulative_pop <= tier_split[2], 'Tier' + "{}".format(year)] = 2
    # Tier 3
    df.loc[(df['IsUrban'] == 2) & (cumulative_pop <= tier_split[3]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_3 / 4.4
    df.loc[(df['IsUrban'] < 2) & (cumulative_pop <= tier_split[3]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_3 / 5.2
    df.loc[cumulative_pop <= tier_split[3], 'Tier' + "{}".format(year)] = 3
    # Tier 4
    df.loc[(df['IsUrban'] == 2) & (cumulative_pop <= tier_split[4]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_4 / 4.4
    df.loc[(df['IsUrban'] < 2) & (cumulative_pop <= tier_split[4]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_4 / 5.2
    df.loc[cumulative_pop <= tier_split[4], 'Tier' + "{}".format(year)] = 4
    # Tier 5
    df.loc[(df['IsUrban'] == 2) & (cumulative_pop <= tier_split[5]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_5 / 4.4
    df.loc[(df['IsUrban'] < 2) & (cumulative_pop <= tier_split[5]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_5 / 5.2
    df.loc[cumulative_pop <= tier_split[5], 'Tier' + "{}".format(year)] = 5

In [42]:
scenario_name = 'High_Ambition_FemiV1'

In [43]:
messagebox.showinfo('OnSSET', 'Browse to the folder where you want to save the outputs')
df.sort_index(inplace=True)
output_dir = filedialog.askdirectory()
output_dir_results = os.path.join(output_dir, '{}.csv'.format(scenario_name))
df.to_csv(output_dir_results, index=False)